In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install numerapi

In [ ]:
import os
import pathlib
import numpy as np
import pandas as pd
import scipy.stats
import tensorflow as tf
import joblib
from tqdm.notebook import tqdm
import numerapi


In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

NUMERAI_S3_BUCKET_URL = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com"
#read in the example predictions from local storage
#EXAMPLE_PREDS = 'tournament_predictions.csv'
#or downlod the example predictions from Numerai's S3 bucket:
EXAMPLE_PREDS_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_example_predictions_data.csv.xz"

2021-08-07 20:08:32,146 INFO numerapi.utils: starting download
./numerai_dataset_276.zip: 416MB [00:09, 43.1MB/s]                           
2021-08-07 20:08:41,805 INFO numerapi.base_api: unzipping file...


In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

exp_df = pd.read_csv(EXAMPLE_PREDS_URL, index_col=0)
training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 8s, sys: 11.3 s, total: 1min 20s
Wall time: 1min 25s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving FisaGol.csv to FisaGol.csv


In [ ]:
import io
import pandas as pd

df1 = pd.read_csv(io.BytesIO(uploaded['FisaGol.csv']))

In [ ]:
from google.colab import files
uploaded2 = files.upload()

Saving FisaGol_1.csv to FisaGol_1.csv


In [ ]:
import io
import pandas as pd

df2 = pd.read_csv(io.BytesIO(uploaded2['FisaGol_1.csv']))

In [ ]:
from google.colab import files
uploaded3 = files.upload()

Saving FisaGol_0_b.csv to FisaGol_0_b.csv


In [ ]:
import io
import pandas as pd

df3 = pd.read_csv(io.BytesIO(uploaded3['FisaGol_0_b.csv']))

In [ ]:
from google.colab import files
uploaded4 = files.upload()

Saving FisaGol_5.csv to FisaGol_5.csv


In [ ]:
import io
import pandas as pd

df4 = pd.read_csv(io.BytesIO(uploaded4['FisaGol_5.csv']))

In [ ]:
# PREDS ENSAMBLE

In [ ]:
preds = (df1['prediction'].values + df2['prediction'].values)/2

In [ ]:
preds = preds + df3['prediction'].values

In [ ]:
preds = preds + df4['prediction'].values

In [ ]:
preds = preds + exp_df.values

In [ ]:
preds = preds / 5

In [ ]:
# Construct submission dataframe.
tournament_data[PREDICTION_NAME] = preds

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.478636
n000920ed083903f    0.450869
n0038e640522c4a6    0.594762
n004ac94a87dc54b    0.590250
n0052fe97ea0c05f    0.556569
                      ...   
nffe4ab391ccef44    0.487385
nffeb0b1c47ed629    0.401008
nfff811b4acc1dd1    0.584829
nfff9aadb3f355c7    0.484951
nfff9e2a06996582    0.549336
Name: prediction, Length: 1784857, dtype: float64

In [ ]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()


In [ ]:

tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

2021-08-07 20:29:12,568 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-08-07 20:29:12,571 INFO numexpr.utils: NumExpr defaulting to 8 threads.


In [ ]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.538074
n000920ed083903f    0.489020
n0038e640522c4a6    0.546383
n004ac94a87dc54b    0.638631
n0052fe97ea0c05f    0.568844
                      ...   
nffe4ab391ccef44    0.495165
nffeb0b1c47ed629    0.457002
nfff811b4acc1dd1    0.635185
nfff9aadb3f355c7    0.537299
nfff9e2a06996582    0.559398
Name: prediction, Length: 1784857, dtype: float64

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol_X.csv')

!cp FisaGol_X.csv "drive/My Drive/PHOENIXSIGMA/"